Projet : Classification de malware

Objectif : détecter si un fichier exécutable est malveillant ou non à l’aide de modèles de machine learning.

Pipeline :
- choisir 5 modèles classiques de machine learning
- réaliser une étude comparative des modèles
- déployer le meilleur modèle

Fonctionnement :
l’utilisateur dépose un fichier exécutable.
la bibliothèque pefile permet d’ouvrir l’exécutable et d’extraire les features.
ces features sont utilisées comme entrée du modèle pour produire une prédiction.

Entrée : fichier exécutable  
Sortie : prédiction (malware / non malware)


In [1]:
#Importation des bibliotèques 
import pandas as pd
#Les modèles 
from sklearn.tree import DecisionTreeClassifier, export_graphviz #export transforme l’arbre en DOT
from graphviz import Source #Sert à lire l'arbre généré avec export_graphiz 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#Pour les graphes
import matplotlib.pyplot as plt 

In [2]:
#Telechager le dataset 
data = pd.read_csv("/Users/apple/Documents/MasterIADB/IA/lesnoteb/malware/Copie de DatasetmalwareExtrait.csv")

In [3]:
data

,AddressOfEntryPoint,MajorLinkerVersion,MajorImageVersion,MajorOperatingSystemVersion,DllCharacteristics,SizeOfStackReserve,NumberOfSections,ResourceSize,legitimate
0,10407,9,6,6,33088,262144,4,952,1
1,5354,9,6,6,33088,262144,4,952,1
2,58807,9,6,6,33088,262144,4,136490,1
3,25166,9,6,6,33088,262144,4,1940,1
4,70387,9,6,6,33088,262144,4,83098,1
...,...,...,...,...,...,...,...,...,...
137439,123291,11,0,5,33088,1048576,5,81654,0
137440,40000,2,6,1,32768,1048576,8,67624,0
137441,59610,10,0,5,33088,1048576,5,22648,0
137442,51216,2,0,1,0,1048576,8,2216,0


Visualisation

In [4]:
#Pour avoir le nombre de lignes et le nombres de colonnes 
data.shape

(137444, 9)

In [5]:
#Pour voir les stats du dataset
data.describe()

,AddressOfEntryPoint,MajorLinkerVersion,MajorImageVersion,MajorOperatingSystemVersion,DllCharacteristics,SizeOfStackReserve,NumberOfSections,ResourceSize,legitimate
count,1.374440e+05,137444.000000,137444.000000,137444.000000,137444.000000,1.374440e+05,137444.000000,1.374440e+05,137444.000000
mean,1.722186e+05,8.620784,68.731876,5.098738,22301.043436,9.306841e+05,4.997119,2.474766e+05,0.297707
std,3.438014e+06,4.095635,1185.709873,99.437584,15444.753219,5.553175e+05,1.917237,2.129516e+07,0.457252
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000e+00,0.000000
25%,1.272100e+04,8.000000,0.000000,4.000000,320.000000,1.048576e+06,4.000000,2.216000e+03,0.000000
50%,5.300800e+04,9.000000,0.000000,5.000000,33088.000000,1.048576e+06,5.000000,9.640000e+03,0.000000
75%,6.157800e+04,10.000000,6.000000,5.000000,33088.000000,1.048576e+06,5.000000,2.376250e+04,1.000000
max,1.074484e+09,255.000000,28619.000000,36868.000000,49504.000000,3.355443e+07,40.000000,4.294903e+09,1.000000


In [6]:
#Pous voir la structure du dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137444 entries, 0 to 137443
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   AddressOfEntryPoint          137444 non-null  int64
 1   MajorLinkerVersion           137444 non-null  int64
 2   MajorImageVersion            137444 non-null  int64
 3   MajorOperatingSystemVersion  137444 non-null  int64
 4   DllCharacteristics           137444 non-null  int64
 5   SizeOfStackReserve           137444 non-null  int64
 6   NumberOfSections             137444 non-null  int64
 7   ResourceSize                 137444 non-null  int64
 8   legitimate                   137444 non-null  int64
dtypes: int64(9)
memory usage: 9.4 MB


In [7]:
#Division des données en features et label
X = data.drop(columns=["legitimate"])
y = data["legitimate"]

In [8]:
#Séparation en données d'entrainements et de tests 
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=1234)

In [9]:
X_test.shape

(41234, 8)

In [10]:

X_test.head(30).to_csv("test.csv", index=False)



Entrainement avec differents types d'algorithmes de classification 

<center><b>1. Arbre de décision</b></center>


In [11]:

tree = DecisionTreeClassifier()  #Creation du  modele
tree.fit(X_train, y_train)  #Entrainement du modele 



,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [12]:
predict1 = tree.predict(X_train) #Prediction avec les données tests
predict2 = tree.predict(X_test) #Prediciton avec les données d'entrainements

print(f"les predictions :{predict2}")
predict2


les predictions :[1 0 0 ... 1 0 0]


array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [13]:
columnList = X_train.columns.tolist() #Pour extraire la liste des features 
columnList



['AddressOfEntryPoint',
 'MajorLinkerVersion',
 'MajorImageVersion',
 'MajorOperatingSystemVersion',
 'DllCharacteristics',
 'SizeOfStackReserve',
 'NumberOfSections',
 'ResourceSize']

In [14]:
Source(export_graphviz(tree, feature_names=columnList)) #Visualisation de l'arbre de decision avec export_graphviz 

In [15]:
#Evaluation 
print(classification_report(y_test, predict2))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99     29117
           1       0.98      0.98      0.98     12117

    accuracy                           0.99     41234
   macro avg       0.99      0.99      0.99     41234
weighted avg       0.99      0.99      0.99     41234



In [16]:
print(confusion_matrix(y_test, predict2))

[[28848   269]
 [  220 11897]]


<center><b>2. RandomForest</b></center>


In [17]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [18]:
#Prediction 
predict3 = rf.predict(X_test)
predict3

array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [19]:
#Evaluation 
print(classification_report(y_test, predict3))
print("La matrice de confusion")
print(confusion_matrix(y_test, predict3))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99     29117
           1       0.98      0.99      0.99     12117

    accuracy                           0.99     41234
   macro avg       0.99      0.99      0.99     41234
weighted avg       0.99      0.99      0.99     41234

La matrice de confusion
[[28931   186]
 [  164 11953]]


Pour les modèles basés sur distance ou gradient comme (KNN, SVM et la regression logistique), la standardisation est importante car elle evite que les features avec de grandes valeurs dominent complètement le calcul des distances et faussent les résultats.

In [20]:
#Standarisation 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std =scaler.transform(X_test)

<center><b>3.SVM(Support vector machine)</b></center>




In [21]:
#Entrainement
support = SVC()
support.fit(X_train_std, y_train)



,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [22]:
#Prediction 
predict4 = support.predict(X_test_std)
predict4

array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [23]:
#Evaluation 
print(classification_report(y_test, predict4))
print("La matrice de confusion")
print(confusion_matrix(y_test, predict4))


              precision    recall  f1-score   support

           0       0.98      0.97      0.97     29117
           1       0.92      0.94      0.93     12117

    accuracy                           0.96     41234
   macro avg       0.95      0.95      0.95     41234
weighted avg       0.96      0.96      0.96     41234

La matrice de confusion
[[28112  1005]
 [  673 11444]]


 <center><b>4.KNeighborsClassifier</b></center>




In [24]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)

array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [25]:
#Prediction 
predict5 = knn.predict(X_test)
predict5

array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [26]:
#Evaluation 
print(classification_report(y_test, predict5))
print("La matrice de confusion")
print(confusion_matrix(y_test, predict5))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98     29117
           1       0.94      0.96      0.95     12117

    accuracy                           0.97     41234
   macro avg       0.96      0.97      0.96     41234
weighted avg       0.97      0.97      0.97     41234

La matrice de confusion
[[28420   697]
 [  525 11592]]


 <center><b>5.Regression logistique</b></center>


In [27]:
Rlogis = LogisticRegression()

Rlogis.fit(X_train_std, y_train)
# predict6 = Rlogis.predict(X_test)
# predict6

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [28]:
predict6 = Rlogis.predict(X_test_std)
predict6

array([1, 0, 0, ..., 1, 0, 0], shape=(41234,))

In [29]:
#Evaluation 
print(classification_report(y_test, predict6))
print("La matrice de confusion")
print(confusion_matrix(y_test, predict6))


              precision    recall  f1-score   support

           0       0.92      0.97      0.95     29117
           1       0.93      0.80      0.86     12117

    accuracy                           0.92     41234
   macro avg       0.93      0.89      0.90     41234
weighted avg       0.92      0.92      0.92     41234

La matrice de confusion
[[28369   748]
 [ 2390  9727]]


<center><b>Comparaison des modèles</b></center>

On a choisi RandomForest car il presente les meilleurs performances avec le metrique F1-score 


<center><b>Deploiement du modèle</b></center>

In [30]:
import joblib 
joblib.dump(rf, "model.pkl") #Sauvegarde du modéle avec la bibliotheque joblib 

['model.pkl']